In [2]:
import numpy as np
from time import time, sleep
from torch import nn, optim
import torch.nn.functional as F
import torch
import random
from random import randrange, randint
import math
from IPython.display import clear_output
import copy
from collections import deque

In [6]:
# functions

def drawBoard(board): 
    print( " ",  board[6], " | ", board[7], " | ", board[8])
    print("-----------------")
    print( " ", board[3], " | ", board[4], " | ", board[5])
    print("-----------------")
    print( " ", board[0], " | ", board[1], " | ", board[2])

def checkVertical(positions):
    if(  (positions[6]==positions[3]==positions[0]==2)
      or (positions[7]==positions[4]==positions[1]==2)
      or (positions[8]==positions[5]==positions[2]==2)):
        return 2
    if(  (positions[6]==positions[3]==positions[0]==1)
      or (positions[7]==positions[4]==positions[1]==1)
      or (positions[8]==positions[5]==positions[2]==1)):
        return 1
    return 0

def checkHorizontal(positions):
    if(  (positions[6]==positions[7]==positions[8]==2)
      or (positions[3]==positions[4]==positions[5]==2)
      or (positions[0]==positions[1]==positions[2]==2)):
        return 2
    if(  (positions[6]==positions[7]==positions[8]==1)
      or (positions[3]==positions[4]==positions[5]==1)
      or (positions[0]==positions[1]==positions[2]==1)):
        return 1
    return 0

def checkDiagonal(positions):
    if(  (positions[6]==positions[4]==positions[2]==2)
      or (positions[0]==positions[4]==positions[8]==2)):
        return 2
    if(  (positions[6]==positions[4]==positions[2]==1)
      or (positions[0]==positions[4]==positions[8]==1)):
        return 1
    return 0

def checkAnyWin(board):
    if(checkVertical(board)==1 or checkHorizontal(board)==1 or checkDiagonal(board)==1 ):
#         print("Player 1 wins")
        return 1
    if(checkVertical(board)==2 or checkHorizontal(board)==2 or checkDiagonal(board)==2 ):
#         print("Player 2 wins")
        return 2
    return 0    

def play(board, remain, action, turn):
    if(len(remain)>0):
        if(action in remain):
            board[action]=turn
            remain.remove(action)            
        else:
            return -10
        
    return checkAnyWin(board)

def win_checker(board, remain, player):
    b = board
    if(player==1):
        for i in remain:
            b[i]=1
            x = checkAnyWin(b)
            b[i]=0            
            if(x==1):
                return i
    else:
        for i in remain:
            b[i]=2
            x = checkAnyWin(b)
            b[i]=0            
            if(x==2):
                return i
    return -1      

def win_blocker(board, remain, player):
    b = board
    if(player==1):
        for i in remain:
            b[i]=2
            x = checkAnyWin(b)
            b[i]=0            
            if(x==2):
                return i
    else:
        for i in remain:
            b[i]=1
            x = checkAnyWin(b)
            b[i]=0            
            if(x==1):
                return i
    return -1     

def oneHot(board):
    return torch.tensor( [(F.one_hot(torch.tensor(board), num_classes=3) ).tolist()], dtype=torch.float32).flatten()

def makeOneHot(player , pos):
    x = math.floor(random.random()*9)
    a = [0.0]*9
    a[pos] = player
    return a

In [6]:
# define models

p1=deque(maxlen=2500)
p2=deque(maxlen=2500)

training1 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax())
target1 = copy.deepcopy(training1)
mse1 = nn.MSELoss()
opt1 = optim.Adam(training1.parameters(), lr=0.001)
training2 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax())
target2 = copy.deepcopy(training2)
mse2 = nn.MSELoss()
opt2 = optim.Adam(training2.parameters(), lr=0.001)


In [7]:
# training cell 

discount_factor = 0.95
error = 0 # count number or errors made during experience collection
a1w=0
a2w=0
ties=0

#training method:

# 1. train model with 1 epsilon and 0.9999 decay
# 2. models will learn to not to play in same position
# 3. for next phase make epsilon low eg. 0.3 and decay same
# 4. along with that make the first move of each agent random so 
#    exploration will be boosted.
# 5. after 10,000 games, first move will not be random anymore 
#    It will play according to its learned behaviour
# 6. Special Functions;
#    win_checker
#    win_blocker

for t in range(20):
    if(t==0):
        epsilon = 1
        decay = 0.9999
        firstRandom = False
        print("done ", t)
    elif(t==6):
        epsilon = 0.1
        decay = 1
        firstRandom = False      
        print("done ", t)  
    elif(t==8):
        epsilon = 0.5
        decay = 1
        firstRandom = False
        print("done ", t)
    elif(t==10):
        epsilon = 0.3
        decay = 1
        firstRandom = True
        print("done ", t)
    elif(t==13):
        epsilon = 0.3
        decay = 0.9999
        firstRandom = True
        print("done ", t)
    elif(t==18):
        epsilon = 0.2
        decay = 0.9999
        firstRandom = False
        print("done ", t)

    for k in range(16000):
        # experience collection
        board = [0,0,0,0,0,0,0,0,0]
        remain = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        done = False
        first = math.floor(random.random()*2)
        for i in range(9):
    #         print(i)
            reward=0
            a=[None]*5
            a[0]=(board[:])

            if(i==8):
                done=True

            # who plays first. 0 = training1 plays first, 1 = training2 plays first
            if(i%2==first): 
                if(i==0 and firstRandom):  
                    action = random.choice(remain)
                    actions_random = makeOneHot(1, action)
                    for j in range(9):
                        actions[j] = actions_random[j]
                    a[1] = actions[:]
                else:
                    actions = training1(oneHot(board))
                    wc = win_checker(board, remain, 1)
                    wb = win_blocker(board, remain, 1)

                    if(wc!=-1):
                        action = wc
                        actions_random = makeOneHot(1, wc)
                        for j in range(9):
                            actions[j] = actions_random[j]
                        a[1] = actions[:]
                    elif(wb!=-1):
                        action = wb
                        actions_random = makeOneHot(1, wb)
                        for j in range(9):
                            actions[j] = actions_random[j]
                        a[1] = actions[:]
                    else:
                        if(random.uniform(0,1)<epsilon):
                            action = math.floor(random.random()*9)
                            actions_random = makeOneHot(1, action)
                            for j in range(9):
                                actions[j] = actions_random[j]
                            a[1] = actions[:]
                        else:
                            actions = training1(oneHot(board))
                            action = actions.argmax().item()
                            a[1] = actions[:]

                x = play(board, remain, action, 1)

            else:
                if(i==0 and firstRandom):
                    action = random.choice(remain)
                    actions_random = makeOneHot(2, action)
                    for j in range(9):
                        actions[j] = actions_random[j]
                    a[1] = actions[:]
                else:
                    actions = training2(oneHot(board))   

                    wc = win_checker(board, remain, 2)
                    wb = win_blocker(board, remain, 2)

                    if(wc!=-1):
                        action = wc
                        actions_random = makeOneHot(2, wc)
                        for j in range(9):
                            actions[j] = actions_random[j]
                        a[1] = actions[:]
                    elif(wb!=-1):
                        action = wb
                        actions_random = makeOneHot(2, wb)
                        for j in range(9):
                            actions[j] = actions_random[j]
                        a[1] = actions[:]
                    else:

                        if(random.uniform(0,1)<epsilon):
                            action = math.floor(random.random()*9)
                            actions_random = makeOneHot(2, action)
                            for j in range(9):
                                actions[j] = actions_random[j]
                            a[1] = actions[:]
                        else:
                            actions = training2(oneHot(board))   
                            action = actions.argmax().item()
                            a[1] = actions[:]

                x = play(board, remain, action, 2)

            a[2]=(board[:])

            if(x==0):   # if played on not played position
                reward += 1 # reward for not playing on already played position
                a[3]=reward
                a[4]=False
                if(i==8):
#                     print("Tie")
                    ties+=1
                    a[3]=5
                    a[4]=True   
                    if(i%2==first): #reward for tie
                        p2[len(p2)-1][3]=5
                    else:
                        p1[len(p1)-1][3]=5

            if(x==1 or x==2): 
                if(x==1):
                    a1w+=1
                if(x==2):
                    a2w+=1
                done=True
                reward += 20
                a[3]=reward
                a[4]=True
#                 print("Winner is ",x)
                if(i%2==first):
                    p1.append(a)
                else:
                    p2.append(a)
    #             print(a)
                break        
            if(x==-10): # if played on already played position
#                 print("Gameplay error, played on already played position")
                error+=1
                done=True
                a[3]=-20
                a[4]=True
                if(i%2==first):
                    p1.append(a)
                else:
                    p2.append(a)
                break
            if(i%2==first):
                p1.append(a)
            else:
                p2.append(a)
        epsilon*=decay

        if(k%600==0 and k!=0):
            print("Errors in batch of 600 games = ",error)
            print("Agent 1 wins =", a1w)
            print("Agent 2 wins =", a2w)
            print("Ties = ", ties)
            a1w=0
            a2w=0
            ties=0
            error=0
            print("learning")
            samples1 = random.sample(p1, 500)
            samples2 = random.sample(p2, 500)
            for i in samples1:
                if(not i[4]):
                    next_q_value = target1(oneHot(i[2])).max().item()
                    y = i[3] + discount_factor*next_q_value
                else:
                    y = i[3]

                t = training1(oneHot(i[0]))

                y_target = [0]*9
                for j in range(9):
                    if(i[0][j]!=0):
                        y_target[j]=-10
                y_target[i[1].argmax().item()] = y 
                y_target = torch.tensor(y_target, dtype=torch.float32)

                loss = mse1(t , y_target)

                opt1.zero_grad()
                loss.backward()
                opt1.step()  

            for i in samples2: # i is a experience tuple (s a s' r a)
                if(not i[4]):
                    next_q_value = target2(oneHot(i[2])).max().item()
                    y = i[3] + discount_factor*next_q_value
                else:
                    y = i[3]

                t = training2(oneHot(i[0]))

                y_target = [0]*9
                for j in range(9):
                    if(i[0][j]!=0):
                        y_target[j]=-10
                y_target[i[1].argmax().item()] = y 
                y_target = torch.tensor(y_target, dtype=torch.float32)

                loss = mse2(t , y_target)

                opt2.zero_grad()
                loss.backward()
                opt2.step()    

        if(k%1200==0):
            target1 = copy.deepcopy(training1)
            target2 = copy.deepcopy(training2)

done  0
Errors in batch of 600 games =  435
Agent 1 wins = 77
Agent 2 wins = 62
Ties =  27
learning
Errors in batch of 600 games =  442
Agent 1 wins = 69
Agent 2 wins = 57
Ties =  32
learning
Errors in batch of 600 games =  413
Agent 1 wins = 72
Agent 2 wins = 72
Ties =  43
learning
Errors in batch of 600 games =  414
Agent 1 wins = 79
Agent 2 wins = 75
Ties =  32
learning
Errors in batch of 600 games =  394
Agent 1 wins = 98
Agent 2 wins = 70
Ties =  38
learning
Errors in batch of 600 games =  372
Agent 1 wins = 104
Agent 2 wins = 76
Ties =  48
learning
Errors in batch of 600 games =  324
Agent 1 wins = 107
Agent 2 wins = 117
Ties =  52
learning
Errors in batch of 600 games =  339
Agent 1 wins = 102
Agent 2 wins = 106
Ties =  53
learning
Errors in batch of 600 games =  326
Agent 1 wins = 129
Agent 2 wins = 87
Ties =  58
learning
Errors in batch of 600 games =  326
Agent 1 wins = 108
Agent 2 wins = 107
Ties =  59
learning
Errors in batch of 600 games =  343
Agent 1 wins = 100
Agent 2 w

Errors in batch of 600 games =  6
Agent 1 wins = 1
Agent 2 wins = 0
Ties =  593
learning
Errors in batch of 600 games =  7
Agent 1 wins = 1
Agent 2 wins = 3
Ties =  589
learning
Errors in batch of 600 games =  4
Agent 1 wins = 2
Agent 2 wins = 3
Ties =  591
learning
Errors in batch of 600 games =  6
Agent 1 wins = 2
Agent 2 wins = 2
Ties =  590
learning
Errors in batch of 600 games =  2
Agent 1 wins = 1
Agent 2 wins = 3
Ties =  594
learning
Errors in batch of 600 games =  6
Agent 1 wins = 3
Agent 2 wins = 1
Ties =  590
learning
Errors in batch of 600 games =  6
Agent 1 wins = 2
Agent 2 wins = 0
Ties =  592
learning
Errors in batch of 600 games =  7
Agent 1 wins = 0
Agent 2 wins = 7
Ties =  586
learning
Errors in batch of 600 games =  6
Agent 1 wins = 1
Agent 2 wins = 0
Ties =  593
learning
Errors in batch of 600 games =  4
Agent 1 wins = 0
Agent 2 wins = 3
Ties =  593
learning
Errors in batch of 600 games =  2
Agent 1 wins = 0
Agent 2 wins = 0
Ties =  598
learning
Errors in batch of 60

Errors in batch of 600 games =  141
Agent 1 wins = 33
Agent 2 wins = 59
Ties =  367
learning
Errors in batch of 600 games =  162
Agent 1 wins = 35
Agent 2 wins = 44
Ties =  359
learning
Errors in batch of 600 games =  221
Agent 1 wins = 65
Agent 2 wins = 84
Ties =  630
learning
Errors in batch of 600 games =  153
Agent 1 wins = 44
Agent 2 wins = 41
Ties =  362
learning
Errors in batch of 600 games =  156
Agent 1 wins = 35
Agent 2 wins = 54
Ties =  355
learning
Errors in batch of 600 games =  137
Agent 1 wins = 28
Agent 2 wins = 55
Ties =  380
learning
Errors in batch of 600 games =  157
Agent 1 wins = 37
Agent 2 wins = 50
Ties =  356
learning
Errors in batch of 600 games =  138
Agent 1 wins = 37
Agent 2 wins = 40
Ties =  385
learning
Errors in batch of 600 games =  150
Agent 1 wins = 45
Agent 2 wins = 46
Ties =  359
learning
Errors in batch of 600 games =  153
Agent 1 wins = 30
Agent 2 wins = 44
Ties =  373
learning
Errors in batch of 600 games =  145
Agent 1 wins = 40
Agent 2 wins = 4

Errors in batch of 600 games =  224
Agent 1 wins = 103
Agent 2 wins = 119
Ties =  154
learning
Errors in batch of 600 games =  213
Agent 1 wins = 118
Agent 2 wins = 92
Ties =  177
learning
Errors in batch of 600 games =  235
Agent 1 wins = 107
Agent 2 wins = 106
Ties =  152
learning
Errors in batch of 600 games =  240
Agent 1 wins = 111
Agent 2 wins = 106
Ties =  143
learning
Errors in batch of 600 games =  222
Agent 1 wins = 124
Agent 2 wins = 96
Ties =  158
learning
Errors in batch of 600 games =  225
Agent 1 wins = 119
Agent 2 wins = 93
Ties =  163
learning
Errors in batch of 600 games =  207
Agent 1 wins = 139
Agent 2 wins = 79
Ties =  175
learning
Errors in batch of 600 games =  228
Agent 1 wins = 107
Agent 2 wins = 88
Ties =  177
learning
Errors in batch of 600 games =  229
Agent 1 wins = 129
Agent 2 wins = 75
Ties =  167
learning
Errors in batch of 600 games =  225
Agent 1 wins = 124
Agent 2 wins = 82
Ties =  169
learning
Errors in batch of 600 games =  227
Agent 1 wins = 139
Ag

Errors in batch of 600 games =  97
Agent 1 wins = 107
Agent 2 wins = 89
Ties =  307
learning
Errors in batch of 600 games =  81
Agent 1 wins = 123
Agent 2 wins = 101
Ties =  295
learning
Errors in batch of 600 games =  80
Agent 1 wins = 123
Agent 2 wins = 100
Ties =  297
learning
Errors in batch of 600 games =  83
Agent 1 wins = 114
Agent 2 wins = 96
Ties =  307
learning
Errors in batch of 600 games =  75
Agent 1 wins = 133
Agent 2 wins = 100
Ties =  292
learning
Errors in batch of 600 games =  71
Agent 1 wins = 124
Agent 2 wins = 109
Ties =  296
learning
Errors in batch of 600 games =  71
Agent 1 wins = 121
Agent 2 wins = 97
Ties =  311
learning
Errors in batch of 600 games =  57
Agent 1 wins = 118
Agent 2 wins = 94
Ties =  331
learning
Errors in batch of 600 games =  109
Agent 1 wins = 221
Agent 2 wins = 165
Ties =  505
learning
Errors in batch of 600 games =  53
Agent 1 wins = 129
Agent 2 wins = 99
Ties =  319
learning
Errors in batch of 600 games =  39
Agent 1 wins = 138
Agent 2 wi

Errors in batch of 600 games =  0
Agent 1 wins = 179
Agent 2 wins = 100
Ties =  321
learning
Errors in batch of 600 games =  0
Agent 1 wins = 161
Agent 2 wins = 97
Ties =  342
learning
Errors in batch of 600 games =  1
Agent 1 wins = 164
Agent 2 wins = 112
Ties =  323
learning
Errors in batch of 600 games =  2
Agent 1 wins = 178
Agent 2 wins = 91
Ties =  329
learning
Errors in batch of 600 games =  1
Agent 1 wins = 181
Agent 2 wins = 97
Ties =  321
learning
Errors in batch of 600 games =  1
Agent 1 wins = 178
Agent 2 wins = 105
Ties =  316
learning
Errors in batch of 600 games =  0
Agent 1 wins = 186
Agent 2 wins = 98
Ties =  316
learning
Errors in batch of 600 games =  0
Agent 1 wins = 151
Agent 2 wins = 108
Ties =  341
learning
Errors in batch of 600 games =  0
Agent 1 wins = 174
Agent 2 wins = 85
Ties =  341
learning
Errors in batch of 600 games =  0
Agent 1 wins = 162
Agent 2 wins = 110
Ties =  328
learning
Errors in batch of 600 games =  0
Agent 1 wins = 141
Agent 2 wins = 90
Ties

In [13]:
# watch 2 agents play against each other 

board=[0]*9
remain = [0,1,2,3,4,5,6,7,8]
first = math.floor(random.random()*2)
for i in range(9):
    
     # who plays first. 
     # 0 = Agent 1 plays first
     # 1 = Agent 2 plays first
        
    if(i%2==first):
        
        #if else structure to make first move random
        
        if(i==0): 
            action = random.choice(remain)
        else:
            actions = training1(oneHot(board))
            action = actions.argmax().item()

        #if you don't want first move random in playing 
        # uncomment these 2 lines and comment if..else
        
#         actions = training1(oneHot(board))
#         action = actions.argmax().item()
        
        x = play(board, remain, action, 1)
        if(x==-10):
            print("Agent 1 lost tried to play on ", action)
            break
        elif(x==1 or x==2):
            print("someone won")
            break

    else:
        #if else structure to make first move random
        
        if(i==0):
            action = random.choice(remain)
        else:
            actions = training2(oneHot(board))
            action = actions.argmax().item()

        # if you don't want first move random in playing 
        # uncomment these 2 lines and comment if..else
        
#         actions = training2(oneHot(board))
#         action = actions.argmax().item()
        
        x = play(board, remain, action, 2)    
        if(x==-10):
            print("Agent 2 lost tried to play on ", action)
            break
    drawBoard(board)
    sleep(1)
    clear_output()

if(x==0):
    print("Tie")
elif(x==1):
    print("Agent 1 won")
elif(x==2):
    print("Agent 2 won")
drawBoard(board)

tie
  2  |  1  |  2
-----------------
  1  |  1  |  2
-----------------
  1  |  2  |  1


In [16]:
# play against one of the agents 

board=[0]*9
remain = [0,1,2,3,4,5,6,7,8]
first = 1
for i in range(9):
    drawBoard(board)
    
     # who plays first. 
     # 0 = Agent 1 plays first
     # 1 = Agent 2 plays first
        
    if(i%2==first): 
        action = int(input()) - 1
        x = play(board, remain, action, 2)
        if(x==-10):
            print("player 1 lost")
            break
        elif(x==1 or x==2):
            print("p1  won")
            break
        

    else:
        actions = training1(oneHot(board))
        action = actions.argmax().item()
        x = play(board, remain, action, 1)    
        if(x==-10):
            print("agent 1 lost", "tried to play on ", action+1)
            break
        elif(x==1 or x==2):
            print("p2 won")
            break
    sleep(1)
    clear_output()
    sleep(0.1)
    
drawBoard(board)

  1  |  0  |  2
-----------------
  2  |  1  |  1
-----------------
  2  |  1  |  2
p2 won
  1  |  1  |  2
-----------------
  2  |  1  |  1
-----------------
  2  |  1  |  2


In [10]:
torch.save(training1, "training1.x")
torch.save(target1, "target1.x")
torch.save(training2, "training2.x")
torch.save(target2, "target2.x")

In [11]:
training1 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax() )
training1 = torch.load("training1.x", weights_only=False)

target1 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax() )
target1 = torch.load("target1.x", weights_only=False)

training2 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax() )
training2 = torch.load("training2.x", weights_only=False)

target2 = nn.Sequential(
    nn.Linear(27, 40),
    nn.ReLU(),
    nn.Linear(40, 9),
    nn.Softmax() )
target2 = torch.load("target2.x", weights_only=False)